In [2]:
import requests
import json
import pandas as pd
import sys
basepath = '/'.join(os.getcwd().split("/")[:-1])

if basepath not in sys.path:
    sys.path.append(basepath)

from becp import projections
import plotly.express as px

import plotly.graph_objects as go

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:
area = 498637

design_type = 'ApartmentHighRise'

params = {
    'state': 'NY',
    'climate_zone': '4A',
    'projection_case': 'MidCase',
    'design_areas': [
        # {
        #     'type': design_type,
        #     'area': area,
        #     'heating_fuel': 'natural gas',
        #     'dhw_fuel': 'natural gas',
        #     'heating_cop': 0.8,
        #     'dhw_cop': 0.8,
        #     'ashrae_standard': 'STD2016'
        # },     
        {
            'type': 'OfficeLarge',
            'area': area,
            'heating_fuel': 'natural gas',
            'dhw_fuel': 'natural gas',
            'heating_cop': 0.8,
            'dhw_cop': 0.8,
            'ashrae_standard': 'STD2016'
        },    
        ]
    }

host =  'https://akf-becp-pyapi.herokuapp.com/'
host = 'http://127.0.0.1:5001'
endpoint = 'get_projection_from_reference_buildings'
payload = json.dumps(params)

results = requests.get(f'{host}/{endpoint}/?params={json.dumps(params)}').json()

pd.DataFrame(results['sizing']['sizing_btuh_per_sf']).sum()

# pd.DataFrame(results['enduses']['enduses_absolute_kbtu']).groupby('enduse').sum()



component                   PeopleLightsEquipmentWater Use EquipmentInfilt...
cooling_sensible_btuh_sf                                            13.516601
cooling_latent_btuh_sf                                               0.636295
heating_btuh_sf                                                      3.899998
dtype: object

In [65]:
from becp import projections

dfs = projections.compile_reference_building_sizing(params['design_areas'], '4A')
dfs['sizing_btuh_per_sf'].sum()

cooling_sensible_btuh_sf    13.516601
cooling_latent_btuh_sf       0.636295
heating_btuh_sf              3.899998
dtype: float64

heating_sensible_btuh_sf     4303.654034
heating_latent_btuh_sf        279.865576
cooling_sensible_btuh_sf    13634.588034
cooling_latent_btuh_sf       2390.837788
dtype: float64


proj = [
    projections.get_projection_from_reference_buildings({
    'state': 'NY',
    'climate_zone': '4A',
    'projection_case': case[2],
    'design_areas': [
        {
            'type': 'SmallOffice',
            'area': 200000,
            'heating_fuel': case[1],
            'dhw_fuel': case[1],
            'heating_cop': case[0],
            'dhw_cop': case[0],
            'ashrae_standard': '90.1-2016'
        },
        {
            'type': 'MediumOffice',
            'area': 10000,
            'heating_fuel': 'Natural Gas',
            'dhw_fuel': 'Natural Gas',
            'heating_cop': 0.8,
            'dhw_cop': 0.8,
            'ashrae_standard': '90.1-2007'
        },
    ]
}) for case in cases]